In [ ]:
import os
import xmlrpc.client
import pandas as pd
from pathlib import Path

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

In [ ]:
search_fact = ["&", "&", "&",
          ("state", "=", "posted"),
          ("invoice_date", ">=", "2024-04-01"), 
          ("invoice_date", "<=", "2024-04-30"), 
          ("journal_id", "in", [10, 90, 30, 97])]

fact_doc_fields = [
          'name',
          'invoice_date',
          'state',
          'reversed_entry_id',
          'reversal_move_id',
          'move_type',
          ]

fact_doc_new_ids = models.execute_kw(api_db, uid, api_clave, 'account.move', 'search', [search_fact])
fact_doc_new_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [fact_doc_new_ids], {'fields': fact_doc_fields})
fact_doc_new_df = pd.DataFrame(fact_doc_new_json)

fact_doc_new_df.loc[:, 'reversed_entry_id'] = fact_doc_new_df.loc[:, 'reversed_entry_id'].str.get(0).astype('Int64')

fact_doc_new_df

In [ ]:
fact_doc_new_df.loc[fact_doc_new_df['move_type'] == 'out_refund']

In [ ]:
fact_doc_new_df.loc[(fact_doc_new_df['move_type'] == 'out_refund') & (fact_doc_new_df['reversed_entry_id'].isna())]

In [ ]:
NCdf = fact_doc_new_df.loc[(fact_doc_new_df['move_type'] == 'out_refund') & (~fact_doc_new_df['reversed_entry_id'].isna())]
NCdf

In [ ]:
reversed_entry_ids = NCdf['reversed_entry_id']

rev_ids_list = []
for rid in reversed_entry_ids:
    rev_ids_list.append(int(rid))

rev_ids_list

fact_doc_from_reversed_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [rev_ids_list], {'fields': fact_doc_fields})
fact_doc_from_reversed_df = pd.DataFrame(fact_doc_from_reversed_json)
fact_doc_from_reversed_df.loc[:, 'reversal_move_id'] = fact_doc_from_reversed_df.loc[:, 'reversal_move_id'].str.get(0).astype('Int64')

fact_doc_from_reversed_df

In [ ]:
dffull = NCdf.merge(fact_doc_from_reversed_df.loc[fact_doc_from_reversed_df['state'] == 'cancel'], how='right', left_on='id', right_on='reversal_move_id')
df = dffull[['id_x', 'name_x', 'invoice_date_x', 'state_x', 'id_y', 'name_y', 'invoice_date_y',
       'state_y']]
df

In [ ]:
archivo = "Notas de Crédito Vigentes vs Facturas Canceladas"

path = Path.home().joinpath('Desktop').joinpath(archivo +  ".xlsx")

df.to_excel(path)